In [1]:
import re
import pandas as pd
import requests
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import os
import pandas as pd
import re
import networkx as nx
import string
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from blanc import BlancHelp, BlancTune

In [9]:
df=pd.read_csv("raw_text.csv")
df

,text,label
0,The following content is\nprovided under a Cre...,Calculus
1,"In this sequence of segments,\nwe review some ...",Probability
2,The following content is\nprovided under a Cre...,CS
3,The following\ncontent is provided under a Cre...,Algorithms
4,The following\ncontent is provided under a Cre...,Algorithms
...,...,...
855,The following content is\nprovided under a Cre...,Math for Eng.
856,&gt;&gt; [MUSIC] &gt;&gt; DAVID J. MALAN: All ...,Diff. Eq.
857,The following content is\nprovided by MIT Open...,Linear Algebra
858,The following content is\nprovided under a Cre...,AI


# Cleaning the data

In [3]:
def clean_data(x):
    
    x = re.sub("&.+;", '', x)
    x = re.sub("\[.+\]", '', x)
    x = re.sub("[MUSIC]", '', x)
    x = re.sub("-+", '', x)
    
   
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    x = _RE_COMBINE_WHITESPACE.sub(" ", x).strip()
    
    return x
df['Cleaned_Transcript'] = df['text'].apply(clean_data)

In [4]:
df = df.replace('\n','', regex=True)
df = df.replace(r"\'",'', regex=True)

In [5]:
df['text'] [856]

'&gt;&gt; [MUSIC] &gt;&gt; DAVID J. MALAN: All right. So this is CS50 and thisis the end of week 10. So some of you might have seen thisalready, but being circulated of late is an article that I thought Id readan excerpt from and then show you a three minute video that paintsthe same picture. It was really a touching story, Ithought, of this intersection of the real world with genuinely compellinguses of technology. &gt;&gt; So the article was entitled, "A boyoversleeps on train, uses Google Maps to find family 25 years later." And thefirst couple of paragraphs were, "When Saroo was five years old he wentwith his older brother to scrounge for change on a passenger trainin a town about two hours from his small hometown. Saroo became tired and hopped on anearby train where he thought his brother was, then fell asleep. When he woke up he was in Calcutta,nearly 900 miles away. Saroo tried to find his wayback, but he didnt know the name of his hometown. And as a tiny illiterate boy in a vas

In [6]:
df['Cleaned_Transcript'] [856]

'DAVD J. ALAN: All right. o this is 50 and this is the end of week 10. o some of you might have seen this already, but being circulated of late is an article that thought d read an excerpt from and then show you a three minute video that paints the same picture. t was really a touching story, thought, of this intersection of the real world with genuinely compelling uses of technology. o the article was entitled, "A boy oversleeps on train, uses Google aps to find family 25 years later." And the first couple of paragraphs were, "When aroo was five years old he went with his older brother to scrounge for change on a passenger train in a town about two hours from his small hometown. aroo became tired and hopped on a nearby train where he thought his brother was, then fell asleep. When he woke up he was in alcutta, nearly 900 miles away. aroo tried to find his way back, but he didnt know the name of his hometown. And as a tiny illiterate boy in a vast city full of forgotten children he had

In [7]:
x="you\'re"
x = re.sub(r"\'", "\'", x)
x

"you're"

In [8]:
df=df.drop(['label','text'],axis=1)


In [4]:
def tokenize(document):
   
    doc_tokenizer = PunktSentenceTokenizer()
    sentences_list = doc_tokenizer.tokenize(document)
    return sentences_list

def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [5]:
def summarise(text):
   
    document = str(text)
    
     
    sentences_list = tokenize(document)
    cv = CountVectorizer()
    cv_matrix = cv.fit_transform(sentences_list)
    cv_demo = CountVectorizer()
  
    
    normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
    res_graph = normal_matrix * normal_matrix.T
    nx_graph = nx.from_scipy_sparse_matrix(res_graph)
    ranks = nx.pagerank(nx_graph)

    sentence_array = sorted(((ranks[i], s) for i, s in enumerate(sentences_list)), reverse=True)
    sentence_array = np.asarray(sentence_array)
    
    rank_max = float(sentence_array[0][0])
    rank_min = float(sentence_array[len(sentence_array) - 1][0])
    temp_array = []

    flag = 0
    if rank_max - rank_min == 0:
        temp_array.append(0)
        flag = 1
    
  
    if flag != 1:
        for i in range(0, len(sentence_array)):
            temp_array.append((float(sentence_array[i][0]) - rank_min) / (rank_max - rank_min))
    
    

    
    threshold = (sum(temp_array) / len(temp_array)) + 0.2
    sentence_list = []
    if len(temp_array) > 1:
        for i in range(0, len(temp_array)):
            if temp_array[i] > threshold:
                    sentence_list.append(sentence_array[i][1])
    else:
        sentence_list.append(sentence_array[0][1])
    
   
    summary = " ".join(str(x) for x in sentence_list)
    return summary

In [18]:
df['Summary']=df['Cleaned_Transcript'].apply(summarise)

# Evaluation

In [3]:
df=pd.read_csv("cleanedLatest.csv")
df

,Unnamed: 0,Cleaned_Transcript,Summary
0,0,The following content is provided under a reat...,"And for now, will just to tell you that the ra..."
1,1,"n this sequence of segments, we review some ma...","What we will talk about is sets, various defin..."
2,2,The following content is provided under a reat...,"o you, as the programmer of this class, are go..."
3,3,The following content is provided under a reat...,ADENE: ADENE: ts a thats an n. PROFEOR: o we w...
4,4,The following content is provided under a reat...,ADENE: n to the 1/8. ADENE: n to the 1/4. ADEN...
...,...,...,...
855,855,The following content is provided under a reat...,o what m going to say is that in any set of n ...
856,856,DAVD J. ALAN: All right. o this is 50 and this...,But the point here is that DN that weve been t...
857,857,The following content is provided by T Openour...,"This c is showing up in the denominator, and t..."
858,858,The following content is provided under a reat...,"o the thing is, that, lets say, you have some ..."


In [4]:
df['Evaluation_Summary']=np.nan

In [5]:
blanc_help = BlancHelp()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
fp=open('evalFinal.txt', 'r') 

In [6]:
fp.read()

',0.20258166491043203,0.2791327913279133,0.17832167832167833,0.20230263157894737,0.22486187845303868,0.12369791666666667,0.314453125,0.11627906976744186,0.10250724884871226,0.37383177570093457,0.34679334916864607,0.12439613526570048,0.11916167664670659,0.28832951945080093,0.30743243243243246,0.3509933774834437,0.16977302085255583,0.12001287001287002,0.1588235294117647,0.14877161055505003,0.2208067940552017,0.16594202898550725,0.15392038600723765,0.15290412876137158,0.19445843828715365,0.24629324546952225,0.23784261715296198,0.3391812865497076,0.1342281879194631,0.1196039603960396,0.20439292251372787'

In [7]:
s=[0.20258166491043203,0.2791327913279133,0.17832167832167833,0.20230263157894737,0.22486187845303868,0.12369791666666667,0.314453125,0.11627906976744186,0.10250724884871226,0.37383177570093457,0.34679334916864607,0.12439613526570048,0.11916167664670659,0.28832951945080093,0.30743243243243246,0.3509933774834437,0.16977302085255583,0.12001287001287002,0.1588235294117647,0.14877161055505003,0.2208067940552017,0.16594202898550725,0.15392038600723765,0.15290412876137158,0.19445843828715365,0.24629324546952225,0.23784261715296198,0.3391812865497076,0.1342281879194631,0.1196039603960396,0.20439292251372787]
len(s)

31

In [22]:
blanc_help = BlancHelp()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for i in range(600,800):
  document = df['Cleaned_Transcript'][i]
  summary = df['Summary'][i]
  val=blanc_help.eval_once(document,summary)
  val=str(val)
  print(val)
  with open('evalFinal.txt', 'a') as writefile:
    writefile.write(",")
    writefile.write(val)

100%|██████████| 1756/1756 [1:20:07<00:00,  2.74s/it]


0.20258166491043203


100%|██████████| 382/382 [14:45<00:00,  2.32s/it]


0.2791327913279133


100%|██████████| 568/568 [27:09<00:00,  2.87s/it]


...

100%|██████████| 1676/1676 [54:15<00:00,  1.94s/it]


0.1196039603960396


100%|██████████| 1800/1800 [56:45<00:00,  1.89s/it]


0.20439292251372787


 68%|██████▊   | 1362/2010 [43:17<19:16,  1.78s/it] 

In [8]:
c=0
for i in range(600,600+len(s)):
    df['Evaluation_Summary'][i]=s[c]
    c=c+1
print(c)
    

31


<ipython-input-8-551a2685e2a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Evaluation_Summary'][i]=s[c]


In [9]:
df=df.dropna()

In [10]:
df

,Unnamed: 0,Cleaned_Transcript,Summary,Evaluation_Summary
600,600,The following content is provided under a reat...,"And then we multiply with this probability, th...",0.202582
601,601,Lets keep building our table of Laplace transf...,Lets make u prime is equal to well do our defi...,0.279133
602,602,Hello everyone. o far in the series on data st...,o with this directed edge represented as order...,0.178322
603,603,"ay have some matrix a lets say a is n by n, so...",o if a has two rows that are equal so if row i...,0.202303
604,604,"n this lesson, were going to write code to del...",f only the left child is null then what want t...,0.224862
605,605,All right. o lets get started. o were still ta...,Eaves Law says this is the expected value of t...,0.123698
606,606,n the last video we defined a transformation t...,o this vector right here is just going to be t...,0.314453
607,607,"n our previous lesson, we talked about level o...",this will work fine in or ++ actually in make ...,0.116279
608,608,The following content is provided under a reat...,ts the probability that Y is in A. That means ...,0.102507
609,609,The following content is provided under a reat...,o to find a method to change the color of the ...,0.373832


In [20]:
df2=pd.read_csv("EvaluateSummaries.csv")
df2

,Unnamed: 0,Unnamed: 0.1,Cleaned_Transcript,Summary,Evaluation_Summary
0,0,0,The following content is provided under a reat...,"And for now, will just to tell you that the ra...",0.199151
1,1,1,"n this sequence of segments, we review some ma...","What we will talk about is sets, various defin...",0.340426
2,2,2,The following content is provided under a reat...,"o you, as the programmer of this class, are go...",0.174203
3,3,3,The following content is provided under a reat...,ADENE: ADENE: ts a thats an n. PROFEOR: o we w...,0.208239
4,4,4,The following content is provided under a reat...,ADENE: n to the 1/8. ADENE: n to the 1/4. ADEN...,0.189708
...,...,...,...,...,...
85,85,626,Lets say that have some set V that is a subspa...,"You could call this the image, because the tra...",0.237843
86,86,627,Lets do some more nonhomogeneous equations. o ...,Weve got the 1 e to the 4x plus 2e to the minu...,0.339181
87,87,628,The last time spent solving a system of equati...,"The characteristic equation, then, is going to...",0.134228
88,88,629,The following content is provided under a reat...,"And if we stop it at b, then the area were int...",0.119604


In [17]:
df2 = df2.drop(df2.columns[[1, 2]], axis=1)

In [21]:
df2['Evaluation_Summary'].mean()

0.19781387168553854

In [29]:

round(0.19781387168553854,2)

0.2

In [12]:
df2=df2.append(df)

In [13]:
len(df2)

90

In [19]:
df2.to_csv('EvaluateSummaries.csv')

In [ ]:
for i in range(207,401):
  document = df['Cleaned_Transcript'][i]
  summary = df['Summary'][i]
  val=blanc_help.eval_once(document,summary)
  val=str(val)
  print(val)
  with open('evalFinal.txt', 'a') as writefile:
    writefile.write(",")
    writefile.write(val)

100%|██████████| 348/348 [09:41<00:00,  1.67s/it]


0.35454545454545455


100%|██████████| 356/356 [11:57<00:00,  2.01s/it]


0.3622994652406417


100%|██████████| 2754/2754 [1:34:00<00:00,  2.05s/it]  


...

100%|██████████| 3556/3556 [1:57:03<00:00,  1.98s/it]  


0.11465073834411814


100%|██████████| 510/510 [20:06<00:00,  2.36s/it]


0.23617021276595745


 65%|██████▍   | 1268/1954 [44:00<32:07,  2.81s/it] 

In [33]:
df['Cleaned_Transcript'][0]

'The following content is provided under a reative ommons license. Your support will help T OpenourseWare continue to offer high quality educational resources for free. To make a donation, or to view additional materials from hundreds of T courses, visit T OpenourseWare at ocw.mit.edu. PROFEOR: o, Professor Jerison is relaxing in sunny London, Ontario today and sent me in as his substitute again. m glad to the here and see you all again. o our agenda today: he said that hed already talked about power series and Taylors formula, guess on last week right, on Friday? o m going to go a little further with that and show you some examples, show you some applications, and then have this course evaluation survey that ll hand out in the last 10 minutes or so of the class. also have this handout that he made that says 18.01 end of term 2007. f you didnt pick this up coming in, grab it going out. People tend not to pick it up when they walk in, see. o grab this when youre going out. Theres some t

In [32]:
df['Summary'][0]

'And for now, will just to tell you that the radius of convergence of this power series is infinity; that is, this sum always converges for any value of x. ll say a little more about that in a few minutes. o, in order to do that, have to know what the derivative of e^x is, and what the second derivative of e^x is, and so on, because that comes into the Taylor formula for the coefficients. The power series expansion of this is going to be a function of t, right, because the variable here is t. get it by taking my expansion for e^x and putting in what x is in terms of t. Whoops! o, to compute the power series for x sin(x), of just take this one and multiply it by x. o lets see if can do that right. o m saying that inside of this radius of convergence, the function that were looking at, this f(x), can be written as the value of the function at 0, thats a_0, plus the value of the derivative. And the way ll do that is by taking the power series expansion for e^x, which we have up there, and

In [34]:

document = df['Cleaned_Transcript'][0]
summary = df['Summary'][0]
blanc_help = BlancHelp()
blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)
print(blanc_help.eval_once(document, summary))
print(blanc_tune.eval_once(document, summary))


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

0.1991507430997877


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-b3c493f94ab9>", line 7, in <module>
    print(blanc_tune.eval_once(document, summary))
  File "C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\blanc\blanc.py", line 102, in eval_once
    (doc_score,) = self.eval_summaries_for_docs([doc], [[summary]])
  File "C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\blanc\blanc.py", line 655, in eval_summaries_for_docs
    self.finetune(finetuned_model, summary)
  File "C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\blanc\blanc.py", line 780, in finetune
    loss.backward()
  File "C:\Users\farhe\anaconda3\envs\CapClone\lib\site-packages\torch\tensor.py", line 185, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\farhe\anaconda3\envs\

TypeError: object of type 'NoneType' has no len()

In [12]:
documents=[]
summaries=[]
for i in range(200,400):
    documents.append(df['Cleaned_Transcript'][i])
    summaries.append(df['Summary'][i])


In [ ]:
blanc_help = BlancHelp()
results=blanc_help.eval_pairs(documents, summaries)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  5%|▌         | 16191/320928 [8:58:18<194:51:27,  2.30s/it]

In [ ]:
results

In [5]:
df['Cleaned_Transcript'][0]

'The following content is provided under a reative ommons license. Your support will help T OpenourseWare continue to offer high quality educational resources for free. To make a donation, or to view additional materials from hundreds of T courses, visit T OpenourseWare at ocw.mit.edu. PROFEOR: o, Professor Jerison is relaxing in sunny London, Ontario today and sent me in as his substitute again. m glad to the here and see you all again. o our agenda today: he said that hed already talked about power series and Taylors formula, guess on last week right, on Friday? o m going to go a little further with that and show you some examples, show you some applications, and then have this course evaluation survey that ll hand out in the last 10 minutes or so of the class. also have this handout that he made that says 18.01 end of term 2007. f you didnt pick this up coming in, grab it going out. People tend not to pick it up when they walk in, see. o grab this when youre going out. Theres some t

In [ ]:
df['Golden_Truth_Summary'][0]='''o, power series is a way of writing a function as a sum of integral powers of x.Theres a number which ll call R, where R can be between 0 and it can also be infinity. ts a number between 0 and infinity, inclusive, so that when the absolute value of x is less than R. o when x is smaller than R in size, the sum converges. This sum that sum converges to a finite value. And when x is bigger than R in absolute value, the sum diverges. This R is called the radius of convergence.Lets compute what will just repeat for you the computation of the power series for e^x, just because its such an important thing to do. o, in order to do that, have to know what the derivative of e^x is, and what the second derivative of e^x is, and so on, because that comes into the Taylor formula for the coefficients. Oh yes, and by the way, whenever you write out a power series, you should say what the radius of convergence is. '''

In [30]:
df.to_csv('cleanedLatest.csv')

In [86]:
df=df[430:860]
df

,Cleaned_Transcript
430,The following content is provided under a reat...
431,OPERATOR: The following content is provided un...
432,The following content is provided under a reat...
433,"tanford niversity. All right, hello everybody ..."
434,We have already seen an example in which we ha...
...,...
855,The following content is provided under a reat...
856,DAVD J. ALAN: All right. o this is 50 and this...
857,The following content is provided by T Openour...
858,The following content is provided under a reat...


In [87]:
df.to_csv("cleanedHalfSecond.csv")

In [4]:
#C:/Users/farhe/Downloads/file-result.csv
dfFinalPart1=pd.read_csv("C:/Users/farhe/Downloads/part1.csv")
dfFinalPart2=pd.read_csv("C:/Users/farhe/Downloads/part2.csv")

In [13]:
dfFinalPart2

,Cluster Level 1,undefined,Cleaned_Transcript
0,Derivative of Y with Respect to X,436,The following content is provided under a reat...
1,Derivative of Y with Respect to X,447,The following content is provided under a reat...
2,Derivative of Y with Respect to X,449,The following content is provided under a reat...
3,Derivative of Y with Respect to X,450,"OK, this is the lecture on linear transformati..."
4,Derivative of Y with Respect to X,454,Well now move from the world of first order di...
...,...,...,...
1497,Other topics,797,We end this lecture sequence by stepping back ...
1498,Other topics,817,"n our previous lessons, we described linked li..."
1499,Other topics,818,We will now define the notion of a random vari...
1500,Other topics,821,The following content is provided under a reat...


In [10]:
df=dfFinalPart1.append(dfFinalPart2, ignore_index = True)

In [19]:
df=df.rename(columns={"Cluster Level 1": "Cluster Level", "undefined": "Transcript Number"})

In [23]:
df

,Cluster Level,Transcript Number,Cleaned_Transcript
0,Question is Okay,29,The following content is provided under a reat...
1,Question is Okay,45,"tanford niversity Hi, everyone. Welcome to the..."
2,Question is Okay,48,o we were talking about the beta distribution ...
3,Question is Okay,50,The real topic is how to solve inhomogeneous s...
4,Question is Okay,62,tanford niversity. am very excited to introduc...
...,...,...,...
2964,Other topics,797,We end this lecture sequence by stepping back ...
2965,Other topics,817,"n our previous lessons, we described linked li..."
2966,Other topics,818,We will now define the notion of a random vari...
2967,Other topics,821,The following content is provided under a reat...


In [22]:
df.to_csv("TopicsLingo.csv")

In [ ]:
dfFinal=pd.read_csv("C:/Users/farhe/Downloads/part1.csv")

# Test codes

In [69]:
!pip install xlsxwriter

In [88]:
import xlsxwriter
import csv

wb = xlsxwriter.Workbook('cleanedHalfSecond.xlsx')
sh = wb.add_worksheet('gkz')

with open('cleanedHalfSecond.csv', 'r') as f:
    reader = csv.reader(f)
    for r, row in enumerate(reader):
        for c, val in enumerate(row):
            sh.write(r, c, val)

wb.close()

In [17]:
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lex_rank import LexRankSummarizer

In [25]:
document='''"it just reminding us of where we started this course we're and what we've been reinforcing throughout the five questions the five filters that you use to figure out whether an opportunity is a good opportunity burning need big enough opportunity scalable business model sustainable differentiated positioning and then why now and why us you really want to find a small segment of the market whose hair is on fire and really needs your product whether it's consumer or business it's very hard to sell something into a market when your start up company it's it's really hard if the people don't have their hair on fire right the old if it's a vitamin versus a pain killer the second point is a big enough market and you want to make sure that well you understand is a total addressable market there is some smaller part of the market that you are going to dominate that base camp whose hair is on fire right we're your product meets that market need hand in glove and these are the things that need to come across to the present in your presentations later this week and hopefully tonight if they don't tonight will be yelling at you enough so that they'll come come cross later this week so again right many of you will have a slide that shows some total market but then the service market but smaller and then initial base camp where everyone's head is going to be nodding in the audience and yeah from what you just described that you do I can understand why that group of people who really needs that product and will buy it my going quickly enough Mike through this thank you scalable business model we've been through this a lot I think you've you've all learned a bit about understanding unit economics the of of one transaction what is a schematic look like of one transaction and then one of the teams in my office hours we talked about amber once you have the unit economics of one sale then you try to get the unit economics of bigger units so for this particular cut for this particular business once they understood that the unit economics of one sale what's going to be pop was gonna be good then they decided Hey let's I decided for them let's look at one market what are the unit economics of one market to you and what are the unit economics of one market to your partner is there enough profit in it for everybody if there is this is a very good business take a look if it's not rethink something because if you're going to enter into a new market you want back to look good to you and you wanted to look into your partner this is an example of one of those business model schematics of where the market we're the product flows in where the the the the dollars flow you should try to have dollars the actual dollars on slides like this this is an example of a slide that it just keep trying to figure out how to get the the servicing the printed out but sustainable differentiated positioning if in your final presentation people are asking a lot about sustainability Hey how can you keep your market once you get there you should feel good because at least they're believing that you can get there right that's a good problem to have right but it is important that you have an answer to that question right what is so novel about what you're doing or how you're doing it that will lead you to be the lasting successful player in that market right questions like what if Google decides to do this are typical of staying ability questions okay so you have to show that differentiation this was the way that consider show their differentiation compared to some of their other products right it can be a crime it can be a list of the most important things about the product and then the different players and and pluses and minuses in numbers for it can be one of those two by two matrices are two by two graphs we're the axes are the most important thing right it's very important that you figure out what those two differentiators are and why you of course wind up in the upper right you also in and showing your business model and how you're going to stay in and be successful we want to show a graph of your financials and you want to show how the what the drivers of those financials are what is going on but make those financial successful in each of your business is good is going to be different this is in terms of differentiation we talked about this earlier as well which is for some of you the differentiator is going to be the technology for some of you the differentiator is going to be how you're approaching the market which includes the special purpose vehicles you may be setting up for the I mean those of you who already have the beginnings of relationships with the hotels that you're serving that will also potential B. the fund be the funding vehicles and you do you'll be developing expertise and how to set up those special purpose vehicles that are going to make this successful that's a differentiator right for some of you it's a technology for some of you which the market for some of you which the financial aspects of things and you want to be up in the right hand side with high market in technology differentiation if you can be and the last question is why you and why now what is going on about your team call if you have this I believe there's something about the people on your team that make you well suited to pursue this opportunity let people understand that and if you've already gotten some advisors and there's some other people on your team that aren't in the class include them on your team slide also and then wind now what is something that's going on with the market with technology some thing going on that surrounds your idea it's going on in the market place in some way that we're we're we're an investor realize Hey you know something that's why this isn't a good time to start this business questions you often hear from investors are why hasn't someone done this already and in answer of well I guess I just didn't think of it is it may be V. it may be the right answer but it's not a very thoughtful one right right off in a special okay yeah I understood I thought mine was an understatement but yeah that that is to for many of your businesses there are regulatory changes that are that are reason for why now right for some of you three seven years behavioral changes wearable devices are becoming something that is is commonplace some product innovation it doesn't have to be your product it's even better if it's not your product somebody else is just innovated something else that's now out there I know somebody who started a company that was using Google glass is the basis for what they were doing now you should probably ride on the coattails of innovation that makes sense but you know but somebody else has an innovation and no no no it's a judgmental one but I don't know that it's political I'm sorry and also even market discontinuities is another good thing that's happening right is the the cost of power under not to an island right or you know some change in the nature of micro grid technology or something that allows you to the the market to change all right and then lastly we don't we haven't pushed us too much in this course but you've all lived this you're doing this very well which is it's not just those generic filters that we've spent most of our time on because that's how you can teach a class but it's that you should each have your own personal filter and when you think there are a lot of when you think of like your own center of gravity people talk about words like authenticity distinctiveness you can consider your strengths that are broadly defined not strange meaning I'm good at accounting or I'm good at marketing I'm good at building things good with people I'm good at leading people I'm good at managing complex projects I'm good it I'm good at keeping cool when everything is going crazy I I really like working with consumer types of businesses I really like working I don't like working with consumer types of businesses I like business to business I like businesses that are manageable market risk where I can grow with fifty percent here forever worth or ventrally little slower you know why don't I like businesses that are platform businesses where I'm developing developing dropping and sometimes it just goes like that all of you were different and you should get a sense of what you enjoy what you like to do and that will drive the type of business across many factors of what makes sense for you different from what makes sense for everybody which is a five filters"
'''
parser = PlaintextParser.from_string(document, Tokenizer("english"))
summarizer = LexRankSummarizer()

summary = summarizer(parser.document, 1) #Summarize the document with 5 sentences

for sentence in summary:
    print(sentence)

"it just reminding us of where we started this course we're and what we've been reinforcing throughout the five questions the five filters that you use to figure out whether an opportunity is a good opportunity burning need big enough opportunity scalable business model sustainable differentiated positioning and then why now and why us you really want to find a small segment of the market whose hair is on fire and really needs your product whether it's consumer or business it's very hard to sell something into a market when your start up company it's it's really hard if the people don't have their hair on fire right the old if it's a vitamin versus a pain killer the second point is a big enough market and you want to make sure that well you understand is a total addressable market there is some smaller part of the market that you are going to dominate that base camp whose hair is on fire right we're your product meets that market need hand in glove and these are the things that need to

In [ ]:
from transformers import pipeline

# using pipeline API for summarization task
summarization = pipeline("summarization")

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
original_text = """okay semantic analysis so %HESITATION once a certain structures understood %HESITATION we can try to understand the meaning okay what this thing actually means and as I started to say before %HESITATION psychologists and other people say the brain of a very good idea lexical analysis works they have a very good idea how pricing works and they basically have no idea how the rest of it works okay so what happens after that it's kind of it's a big mystery events and that's one of the pride of the outstanding mysteries actually it %HESITATION that remains %HESITATION and compilers would certainly love it was really great of compilers correctly understand what your program is supposed to do but that's way too hard we still want to be able to write compiler so we do something much more limited %HESITATION in compilers K. so compilers can't understand the full meaning of your program this has many profound implications actually for how programmers in compilers interact Mrs compiler don't actually know what you're trying what program you're trying to write they can only give you limited help and if you make mistakes they typically can't catch them all all right so you'll you'll do exactly what you tell us to do and if you tell to do the wrong thing that's your problem all right but there are some things are some areas where compilers can kind of get it meaning a little bit and help %HESITATION you too bye and catch problems with your program all right and and basically what compilers do they don't know what your program is supposed to do %HESITATION but they can look for inconsistencies so if your program says do X. and then someplace else you say do not X. K. the compiler can say %HESITATION you know you you told me to do acts and not acts and they can't both be right and so that's some kind of an error in your program all right and so those aspect that's the level of understanding that compilers can do let's do some %HESITATION examples based on English first okay so so Jack's a Jerry left his assignment at home right so how many people think that his refers to Jack how many people think it refers to Jerry how many people think it refers to some third person that we don't see could be right to give you some context here that we don't we don't see that there's actually so it could be it could be two or three people involved here in his could refer to %HESITATION any you know any one of three different people now if I say Jack said Jack left to the same at home well there could be we don't know how many jacks there are all right well we don't know which one left the assignment and again his doesn't even have to refer to one of the two possible jacks I could refer to some some third party that we don't see all right so so is that kind of %HESITATION ambiguity or uncertainty about what names buying two K. %HESITATION is that's that's a big no no in programming language design we design a programming language that is that hard to figure out what it's supposed to what what you know if you're able supposed to mean %HESITATION it's the it would not be a probably be very popular language with most programmers so let me just have programming languages suppose as opposed to natural language have %HESITATION is very strict rules to avoid such problems okay so here's kind of the analogy %HESITATION you know here we have you know two different jacks and then we're printing out we use the name here and so we need a rule that tells us which one we mean okay so it's the meaning of the program is is well the fine and most languages %HESITATION %HESITATION use lexical scope as it's called I'm sure you've seen this in earlier classes and the rule be that this reference here would bind to the inner definition so we would when I say Jack here I get this one and the outer definition is hidden or what's called shadowed meeting that within that in the in the scope of that internet nation I can't even refer to the other one at all all right because they have the same name if I use that name I always get the inner one and the other one is inaccessible at least within that scope all right any questions about that they should be they should be stuffy %HESITATION probably mostly seen before now compilers perform lots of semantic checks besides very able bindings so let's do another simple example so Jack left her homework at home all right so assuming that Jack is mail which is maybe not a hundred percent safe assumption but assuming that we can infer that Jack is is male %HESITATION then there's a type mismatch between Jack and her so you know hers prison which email and so we know there differently you know they're different people okay so in this case you know very likely from reading the sentence we know they're two people involved %HESITATION it and that Jack and her distinct all right ands %HESITATION and type checking %HESITATION is basically the same kind of thing in in programming languages so in addition to just names of variables you very Abel's have types and if two things have different types and we know they can't be the same thing and the compilers were actually rely on that all the time to know that you know I'm definitely referring to different things here that means I need to register is or you know there's no chance if I re order these things I'm changing the order references to them because they were distinct things to begin with and so I know I'm talking bout different memory location right that level of reasoning is actually really really common and used a lot in %HESITATION reading about types %HESITATION in in compilers all right when we do our section on type checking we'll see this in in full detail all right so that's a little bit about semantic analysis """


In [ ]:
summary_text = summarization(original_text)[0]['summary_text']
print("Summary:", summary_text)

In [1]:
import requests
import base64



In [ ]:
from google.cloud import speech

client = speech.SpeechClient()

# path = 'resources/commercial_mono.wav'
with io.open(path, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=8000,
    language_code="en-US",
    # Enable automatic punctuation
    enable_automatic_punctuation=True,
)

response = client.recognize(config=config, audio=audio)

for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    print("-" * 20)
    print("First alternative of result {}".format(i))
    print("Transcript: {}".format(alternative.transcript))

In [6]:
url = "https://proxy.api.deepaffects.com/text/generic/api/v1/async/punctuate"

querystring = {"apikey":"mPt8BQpoVh1EB9i5Diri9hswdFiyXTrv", "webhook":"https://proxy.api.deepaffects.com/text/generic/api/v1/async/punctuate"}

payload = {"texts": ["so its more fluid than it is and you know its not the best kind of feedback right"]}

headers = {
    'Content-Type': "application/json",
}

response = requests.post(url, json=payload, headers=headers, params=querystring)

print(response.text)

{"api":"/text/generic/api/v1/async/punctuate","request_id":"b46b7e66-8637-4512-a642-63dc08feefe2"}


In [15]:
print(dir(response))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [28]:
print(response.text)

{"api":"/text/generic/api/v1/async/punctuate","request_id":"b46b7e66-8637-4512-a642-63dc08feefe2"}


In [29]:

# importing the requests library
import requests
  
url = "http://bark.phon.ioc.ee/punctuator"


In [41]:
headers = {"content-type": "application/json", "Accept-Charset": "UTF-8"}
  
r = requests.post(url, data={"sample":'''I'M hoping it okay, no all right, so I want to develop a certain number of naturally a certain number of properties of the F. T., and I would say well and and our goal actually despite the fact that, but but you don't see visibly anymore a connection To the continuous case, I still want to follow the path of trying to make the D. F. T. look as much as possible, like the continuous for a transfer. My said that a beginning, when it first or talk about this because we buy now, we built up a lot of intuition what a hard won experience operating with a continuous time for a transfer, and I want to leverage that all right. So I want to make the D. F. T. look as much as possible and when I, when I say make it look as much as possible, I mean the properties different different properties that might have general properties formulas. One of my good do you have T. look like the continuous time for a transfer. You don't have to do this, but I really think you're handicapping yourself. If you don't all right now, this is based mostly on the two things. That'Ll help that'll help through this two things that help one is to introduce the so called discrete complex exponential, and the second is to increase the discrete delta's. Okay, those two things: singling out those two things for special attention, special notation, if you want, is, will actually help us write formulas. In the discrete case, it looks very much like the formulas in the continuous case, all right by the way. I have good news here for the discrete delta is no more tests functions, no distributions, it's all, really pretty simple. Okay you'll like that, so let me do each one of the separately. Let me first introduce the discrete complex exponential, explain what I mean by that. It'S just a matter of introducing different Tatian for what's already in front of us. Okay, let me write down the formula for the DFT not affect. Let me write down the form like this once right. The right to do. You have T., as the four entrance run, I'll put even a little under line into the script up, although, like I say, I'm not gon na make a fetish out of this, so it was pretty for a transform of F. if I waited an index and is Of a discrete function, F. is, by definition, the song from an eagle zero up to N. minus one F. from, and when I do, I'm I'm using tax. K. I want for my notes here, because I want to screw myself up in my my notation. Some from cable zero - I ten minus one F. K. either the minus to pile I. N. K. over an all right. That'S the definition of this before I transfer my gon na make it I'm going to to to make it look like a freight transfer. I might as well use the same. Notation is a for a transfer, or at least a slightly modified version of it. Okay, now it's helpful. So what what comes in this morning, of course, to be complex, exponentials so what's helpful, is regard the complex exponential to come into this formula as themselves coming from eight discrete function. So in regard V. either the minus two pie, all right Kay and what about and as themselves coming from discrete function now this is very common, but what's not completely common is a notation for this, so I'm gon na I'm, I'm just I'm, I'm just I'm winters Location for what I mean here, so I'm going to consider the discrete function so let omega and and equals zero up to and minus one B. the discrete function thanks omega and the install you is just either the minus two hi I and over capital, and so I could use a cat, substring capital and here to distinguish the the capital and it's coming up. I I generally won't do that the capital and is understood site all right, so, in other words, if I right I'll make it just in terms of its in terms of its entries, make, it would look like omega. Zero is make a at zero if indexes either zeros one. So but it's it's one, either the minus to pile I over, and you know the minus two pie all by times too, over and all the way up to either the minus to pile I times and minus one divided by an that's the list of values. Okay and then further more, what's not what's what's involved in the definition of the day, if he is not just make about make it to a power all right so also, then, and also make it a minus K. discrete function. Is this either the minus to pile? I and K., via my end, it's just raising rates just raising each one of the entries to the cave. Part of mine is K. power all right, so omega minus K. if I just as just as just a list of values, is one either the minus two pie. All I K. over and either the minus to pile I. two K. over an you know. I got ta, be careful here hold on a second here. I'M sorry. Let me change this. I went I I it's it's it's a question. We put your mind to find my apologies, because I want to put a minus sign there. I want. I want this to be the complex exponential minute. One take negative powers, I'm sorry so Megan is either to pie and over, and so this is a list of powers. It'Ll make it that'll make it a minus. K. is just the powers that occur in the in the D. F. T.. My apologies for for for saying that wrong. First, so then it goes all the way up to either the minus two pie, all bye, two pie, all right and minus one K. divided by Capital, One all right hands the D. F. T. in terms of this complex, exponential, discreet, complex exponential looks like that. So in terms of omega minus, K. you can write the discrete for a transform at and is this from K. going from zero to and minus one of F. K. times may I got to the minus K. evaluated at an if I drop the very able on Both sides drop the very able N. to rights before I transfer of after just before I transfer of F. is the sum from K. going from zero up to N. minus one, F. of K. all may got to the minus K. that's as close as I can Come to making the formula for the disk before I transfer looks like a formula for the continuous for a transformer. It'S a some instead of an integral and it's a discreet, complex, exponential server instead of the continuous complex exponential, but there it is okay. So that's it's! It is you, will, you will see we'll see when we start driving formals, it's useful to be able to write the formula this way to be able to get you seen this before you know. Sometimes you want to be able to write very Abel's right formals without the very Abel's, because sometimes we put very Abel's in the formula as it gets a little dicey. What role the variables are playing. So I'm introducing this extra terminology not to make your life partner. Not to give you more things to think about. But because there are occasions where it's more useful to write the formulas with as few variables as possible and as clean away as possible, and the way to do that is to introduce these complex exponentials as themselves coming from a discrete function, all right. That is great. I have no idea. No, I love answering questions that one happens all too often, no asking network. Okay. No, I don't think it's a Z. transform, but I have no idea, but I don't think so all right now. We will see you next time actually not today, but next time we will see that the interesting properties of the D. F. T. arise from V. or **** allergy of the discrete complex exponentials. That is one of things. We'Re going to see is that the discrete complex expand and we wouldn't it would. It would be more difficult to formulate that those properties - if I didn't introduce this, if I didn't - introduce a couple of exponential as sort of discrete version of this function, all right this discrete function. So we will see that next time that the this is one of the key properties of the D. F. T. its use and and driving all the interesting problem must be interesting. Properties of it is that they just need a discreet, complex, exponentials, R. or fog and then almost or so normal, but not quite or someone'''}, headers=headers)




  
# extracting latitude, longitude and formatted address 
# of the first matching location
print(dir(r))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [42]:
r.text

''

In [1]:
import requests

data = {
    'text': ''' The following content is provided under a reative ommons license Your support will help T OpenourseWare continue to offer high quality educational resources for free To make a donation, or to view additional materials from hundreds of T courses, visit T OpenourseWare at ocwmitedu PROFEOR: o, Professor Jerison is relaxing in sunny London, Ontario today and sent me in as his substitute again m glad to the here and see you all again o our agenda today: he said that hed already talked about power series and Taylors formula, guess on last week right, on Friday? o m going to go a little further with that and show you some examples, show you some applications, and then have this course evaluation survey that ll hand out in the last 10 minutes or so of the class also have this handout that he made that says 1801 end of term 2007 f you didnt pick this up coming in, grab it going out People tend not to pick it up when they walk in, see o grab this when youre going out Theres some things missing from it He has not decided when his office hours will be at the end of term He will have them, just hasnt decided when o, check the website for that information And were looking forward to the final exam, which is uh arent we? Any questions about this technical stuff? All right, lets talk about power series for a little bit o thought should review for you what the story with power series is OK, could have your attention please? o, power series is a way of writing a function as a sum of integral powers of x These a_0, a_1, and so on, are numbers An example of a power series is a polynomial Not to be forgotten, one type of power series is one which goes on for a finite number of terms and then ends, so that all of the other, all the higher a_is are all 0 This is a perfectly good example of a power series; its a very special kind of power series And part of what want to tell you today is that power series behave, almost exactly like, polynomials Theres just one thing that you have to be careful about when youre using power series that isnt a concern for polynomials, and ll show you what that is in a minute o, you should think of them as generalized polynomials The one thing that you have to be careful about is that there is a number o one caution Theres a number which ll call R, where R can be between 0 and it can also be infinity ts a number between 0 and infinity, inclusive, so that when the absolute value of x is less than R o when x is smaller than R in size, the sum converges This sum that sum converges to a finite value And when x is bigger than R in absolute value, the sum diverges This R is called the radius of convergence o well see some examples of what the radius of convergence is in various powers series as well, and how you find it also But, let me go on and give you a few more of the properties about power series which think that professor Jerison talked about earlier o one of them is theres a radius of convergence Heres another one f youre inside of the radius convergence, then the function has all its derivatives, has all its derivatives, just like a polynomial does You can differentiate it over and over again And in terms of those derivatives, the number a_n in the power series can be expressed in terms of the value of the derivative at 0 And this is called Taylors formula o m saying that inside of this radius of convergence, the function that were looking at, this f(x), can be written as the value of the function at 0, thats a_0, plus the value of the derivative This bracket n means you take the derivative n times o when n is 1, you take the derivative once at 0, divided by 1!, which is !, and multiply it by x Thats the linear term in the power series And then the quadratic term is you take the second derivative Remember to divide by 2!, which is 2 ultiply that by x^2 and so on out o, in terms o the coefficients in the power series just record the values of the derivatives of the function at x = 0 They can be computed that way also Lets see think thats the end of my summary of things that he talked about think he did one example, and ll repeat that example of a power series This example wasnt due to David Jerison; it was due to Leonard Euler ts the example of where the function is the exponential function e^x o, lets see Lets compute what will just repeat for you the computation of the power series for e^x, just because its such an important thing to do o, in order to do that, have to know what the derivative of e^x is, and what the second derivative of e^x is, and so on, because that comes into the Taylor formula for the coefficients But we know what the derivative of e^x is, its just e^x again, and its that way all the way down All the derivatives are e^x over and over again o when evaluate this at x = 0, well, the value of e^x is 1, the value of e^x is 1 at x = 0 You get a value of 1 all the way down o all these derivatives at 0 have the value 1 And now, when plug into this formula, find e^x is 1 plus 1*x plus 1/2! x^2 plus 1/3! x^3, plus and so on o all of these numbers are 1, and all you wind up with is the factorials in the denominators Thats the power series for e^x This was a discovery of Leonhard Euler in 1740 or something Yes, aam ADENE: When youre writing out the power series, how far do you have to write it out? PROFEOR: How far do you have to write the power series before it becomes well defined? Before its a satisfactory solution to an exam problem, suppose, is another way to phrase the question ntil you can see what the pattern is can see what the pattern is s there anyone whos in doubt about what the next term might be? ome people would tell you that you have to write the summation convention thing Dont believe them f you right out enough terms to make it clear, thats good enough OK? s that an answer for you? ADENE: Yes, Thank you PROFEOR: OK, so thats a basic example Lets do another basic example of a power series Oh yes, and by the way, whenever you write out a power series, you should say what the radius of convergence is And for now, will just to tell you that the radius of convergence of this power series is infinity; that is, this sum always converges for any value of x ll say a little more about that in a few minutes Yeah? ADENE: o which functions can be written as power series? PROFEOR: Which functions can be written as power series? Thats an excellent question Any function that has a reasonable expression can be written as a power series m not giving you a very good answer because the true answer is a little bit complicated But any of the functions that occur in calculus like sines, cosines, tangents, they all have power series expansions, OK? Well see more examples Lets do another example Heres another example guess this was example one o, this example, think, was due to Newton, not Euler Lets find the power series expansion of this function: 1/(1x) Well, think that somewhere along the line, you learned about the geometric series which tells you that which tells you what the answer to this is, and ll just write it out The geometric series tells you that this function can be written as an alternating sum of powers of x You may wonder where these minuses came from Well, if you really think about the geometric series, as you probably remembered, there was a minus sign here, and that gets replaced by these minus signs think maybe Jerison talked about this also Anyway, heres another basic example Remember what the graph of this function looks like when x = 1 Then theres a little problem here because the denominator becomes 0, so the graph has a pole there t goes up to infinity at x = 1, and thats an indication that the radius of convergence is not infinity Because if you try to converge to this infinite number by putting in x = 1, here, youll have a big problem n fact, you see when you put in x = 1, you keep getting 1 in every term, and it gets bigger and bigger and does not converge n this example, the radius of convergence is 1 OK, so, lets do a new example now Oh, and by the way, should say you can calculate these numbers using Taylors formula f you havent seen it, check it out alculate the iterated derivatives of this function and plug in x = 0 and see that you get 1, 1, 1, 1, and'''
}

response = requests.post('http://bark.phon.ioc.ee/punctuator', data=data)


In [2]:
response.text

'The following content is provided under a reative ommons license. Your support will help T OpenourseWare continue to offer high quality educational resources for free To make a donation or to view additional materials. From hundreds of T courses visit, T OpenourseWare at ocwmitedu, PROFEOR o Professor Jerison, is relaxing in sunny London Ontario today and sent me in as his substitute again m glad to the here and see you all again o our agenda today. He said that hed already talked about power series and Taylors formula, guess on last week, right on Friday, o m going to go a little further with that and show you some examples, show you some applications and then have this course. Evaluation survey that ll hand out in the last 10 minutes or so of the class also have this handout that he made. That says, 1801 end of term 2007. F, you didnt pick this up coming in grab it going out. People tend not to pick it up when they walk in see o grab this. When youre going out Theres some things mis